# Линейная алгебра: сходство текстов и аппроксимация функций

Дан набор предложений, скопированных с Википедии. Каждое из них имеет "кошачью тему" в одном из трех смыслов:

•	кошки (животные)

•	UNIX-утилита cat для вывода содержимого файлов

•	версии операционной системы OS X, названные в честь семейства кошачьих


In [1]:
import numpy as np
import re
from scipy.spatial.distance import cosine

### Считаем файл и переведем текст в нижний регистр

In [2]:
#Считаем файл
with open('sentences.txt', "r") as file:
    s=list(file)
# переводим в нижний регистр
s_lower = []
for i in range(len(s)):
    s_lower.append(s[i].lower())
for line in s_lower:
    print(line)

in comparison to dogs, cats have not undergone major changes during the domestication process.

as cat simply catenates streams of bytes, it can be also used to concatenate binary files, where it will just concatenate sequence of bytes.

a common interactive use of cat for a single file is to output the content of a file to standard output.

cats can hear sounds too faint or too high in frequency for human ears, such as those made by mice and other small animals.

in one, people deliberately tamed cats in a process of artificial selection, as they were useful predators of vermin.

the domesticated cat and its closest wild ancestor are both diploid organisms that possess 38 chromosomes and roughly 20,000 genes.

domestic cats are similar in size to the other members of the genus felis, typically weighing between 4 and 5 kg (8.8 and 11.0 lb).

however, if the output is piped or redirected, cat is unnecessary.

cat with one named file is safer where human error is a concern - one wrong us

### Произведем разбиение текста на слова(токенизацию), используя регулярные выражения, которое считает разделителем любой символ, не являющийся буквой. Удалим пустые слова.

In [3]:
words = []
for line in s_lower:
    words.append(re.split('[^a-z]', line))
words_clear = [[] for i in range(len(words))]

i = 0
for line in words:
    for word in line:
        if word != '':
            words_clear[i].append(word)
    i = i + 1
for line in words_clear:
    print(line)

['in', 'comparison', 'to', 'dogs', 'cats', 'have', 'not', 'undergone', 'major', 'changes', 'during', 'the', 'domestication', 'process']
['as', 'cat', 'simply', 'catenates', 'streams', 'of', 'bytes', 'it', 'can', 'be', 'also', 'used', 'to', 'concatenate', 'binary', 'files', 'where', 'it', 'will', 'just', 'concatenate', 'sequence', 'of', 'bytes']
['a', 'common', 'interactive', 'use', 'of', 'cat', 'for', 'a', 'single', 'file', 'is', 'to', 'output', 'the', 'content', 'of', 'a', 'file', 'to', 'standard', 'output']
['cats', 'can', 'hear', 'sounds', 'too', 'faint', 'or', 'too', 'high', 'in', 'frequency', 'for', 'human', 'ears', 'such', 'as', 'those', 'made', 'by', 'mice', 'and', 'other', 'small', 'animals']
['in', 'one', 'people', 'deliberately', 'tamed', 'cats', 'in', 'a', 'process', 'of', 'artificial', 'selection', 'as', 'they', 'were', 'useful', 'predators', 'of', 'vermin']
['the', 'domesticated', 'cat', 'and', 'its', 'closest', 'wild', 'ancestor', 'are', 'both', 'diploid', 'organisms', 't

### Составим список всех слов, встречающихся в предложениях. Сопоставим каждому слову индекс от нуля до (d - 1), где d — число различных слов в предложениях. Для этого удобно воспользоваться структурой dict.

In [4]:
words_dict = {}

i = 0
for line in words_clear:
    for word in line:
        if word not in words_dict:
            words_dict[word] = i
            i += 1
            
for item in words_dict:
    print(words_dict[item],": ",item)

0 :  in
1 :  comparison
2 :  to
3 :  dogs
4 :  cats
5 :  have
6 :  not
7 :  undergone
8 :  major
9 :  changes
10 :  during
11 :  the
12 :  domestication
13 :  process
14 :  as
15 :  cat
16 :  simply
17 :  catenates
18 :  streams
19 :  of
20 :  bytes
21 :  it
22 :  can
23 :  be
24 :  also
25 :  used
26 :  concatenate
27 :  binary
28 :  files
29 :  where
30 :  will
31 :  just
32 :  sequence
33 :  a
34 :  common
35 :  interactive
36 :  use
37 :  for
38 :  single
39 :  file
40 :  is
41 :  output
42 :  content
43 :  standard
44 :  hear
45 :  sounds
46 :  too
47 :  faint
48 :  or
49 :  high
50 :  frequency
51 :  human
52 :  ears
53 :  such
54 :  those
55 :  made
56 :  by
57 :  mice
58 :  and
59 :  other
60 :  small
61 :  animals
62 :  one
63 :  people
64 :  deliberately
65 :  tamed
66 :  artificial
67 :  selection
68 :  they
69 :  were
70 :  useful
71 :  predators
72 :  vermin
73 :  domesticated
74 :  its
75 :  closest
76 :  wild
77 :  ancestor
78 :  are
79 :  both
80 :  diploid
81 :  organi

### Создадим матрицу размера n * d, 
где n — число предложений. Заполним ее: элемент с индексом (i, j) в этой матрице должен быть равен количеству вхождений j-го слова в i-е предложение. Должна получиться матрица размера 22 * 254.

In [5]:
import pandas as pd

frame = pd.DataFrame(words_dict, range(len(words_clear)))
rows, col = frame.shape
print('Размер фрейма:',rows,'x',col)
for i in range(rows):
    for j in range(col):
        frame.iloc[i, j] = 0
        
#frame

Размер фрейма: 22 x 254


In [6]:
for i in range(len(words_clear)):
    for word in words_clear[i]:
        frame.loc[i, word] += 1     
frame

,in,comparison,to,dogs,cats,have,not,undergone,major,changes,...,disk,run,off,external,drive,fifth,update,features,more,count
0,1,1,1,1,1,1,1,1,1,1,...,0,0,0,0,0,0,0,0,0,0
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,2,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,2,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,1,0,1,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,1,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Найдите косинусное расстояние от предложения в самой первой строке до всех остальных
с помощью функции scipy.spatial.distance.cosine. Какие номера у двух предложений, ближайших к нему по этому расстоянию (строки нумеруются с нуля)? Эти два числа и будут ответами на задание. Само предложение (In comparison to dogs, cats have not undergone... ) имеет индекс 0.

Основная идея, на которой базируется расчет косинусного расстояния, заключается в том, что строку из символов можно преобразовать в числовой вектор. Если проделать эту процедуру с двумя сравниваемыми строками, то меру их сходства можно оценить через косинус между двумя числовыми векторами.

Основным преимуществом косинусного расстояния является то, что данная метрика хорошо работает на разреженных данных (реальные тексты ключевых фраз могут быть очень длинными, содержать значительные объемы служебной информации, такой как минус-слова, стоп-слова и т. д.) Ключевым недостатком косинусного расстояния является его очень сильная зависимость от форм слова.

In [12]:
dist1 = []

for i in range(rows):
    dist1.append(round(cosine(frame.iloc[0], frame.iloc[i]),3))
    
print(dist1)

[0, 0.953, 0.864, 0.895, 0.777, 0.94, 0.733, 0.926, 0.884, 0.906, 0.833, 0.88, 0.84, 0.87, 0.874, 0.944, 0.841, 0.957, 0.944, 0.889, 0.843, 0.825]


In [13]:
d = list(dist1)
ClosestValues = [[-1, 0], [-1, 0]]

d.remove(min(d))

for i in range(2):
    ClosestValues[i][1] = min(d)
    for j in range(len(dist1)):
        if ClosestValues[i][1] == dist1[j]:
            ClosestValues[i][0] = j
    d.remove(min(d))

ClosestValues = sorted(ClosestValues)
ClosestValues

[[4, 0.777], [6, 0.733]]

### Запись ответа

In [9]:
with open('result.txt', 'w') as f:
    for i in range(len(ClosestValues)):
        f.write(str(ClosestValues[i][0]) + ' ')